# ERA5-Land Portugal heat/need exposure indices (2010–2012) — native ERA5-Land grid exports

This notebook computes four indices from **ERA5-Land HOURLY** `skin_temperature` over **Portugal (mainland + Azores + Madeira)**:

- **hdd_18**: count of days where **daily mean** skin temperature < 18°C  
- **cdd_25**: count of days where **daily mean** skin temperature > 25°C  
- **extreme_heat**: mean of the **6th–10th hottest** *daily mean* temperatures  
- **extreme_cold**: mean of the **6th–10th coldest** *daily mean* temperatures  

**Key design choice:** computations stay on the **native ERA5-Land grid** (regular lat/lon) throughout, and final exports are also on that same native grid.

Outputs:
- 4 single-band **GeoTIFFs** exported to your **Google Drive** (Earth Engine export tasks).

> Notes  
> - ERA5-Land `skin_temperature` is provided in **Kelvin**; we convert to **°C** as `K - 273.15`.  
> - Indices are computed per pixel, then clipped to Portugal.  
> - Exports run as asynchronous Earth Engine tasks; monitor them in the Earth Engine Tasks UI or via the optional polling cell.


In [1]:
# (Colab) Install Earth Engine Python API (and helpers) if needed
!pip -q install earthengine-api geemap

In [2]:
import ee
import datetime
import time

# Authenticate & initialize (Colab will prompt you)
ee.Authenticate()
ee.Initialize(project='energy-poverty-from-space')

print("EE initialized:", ee.String('ok').getInfo())

EE initialized: ok


## Parameters

In [3]:
import re

# ---- User parameters ----
DRIVE_FOLDER = "ERA5L_PT_nativegrid_2010_2012"  # Drive folder name to create/use
MAX_PIXELS = 1e13

START_YEAR = 2010
END_YEAR = 2012  # inclusive

# Use DAILY aggregates to avoid hourly explosion.
# (Daily aggregates are already averaged from the hourly ERA5-Land collection.)
ERA5_COLLECTION = "ECMWF/ERA5_LAND/DAILY_AGGR"
TEMP_BAND = "skin_temperature"  # Kelvin (daily mean)

# If you prefer nearest-neighbor behavior (no smoothing), set False.
USE_BILINEAR = False

# --- Load collection (server-side object) ---
era5 = ee.ImageCollection(ERA5_COLLECTION).select(TEMP_BAND)

native_proj = ee.Image(era5.first()).projection()
NATIVE_CRS = native_proj.crs().getInfo()

t = native_proj.transform().getInfo()

if isinstance(t, str):
    # Parse WKT-ish Affine definition into GDAL-style 6 numbers
    params = dict(re.findall(r'PARAMETER\["(elt_\d_\d)",\s*([-\d\.eE]+)\]', t))
    a = float(params["elt_0_0"])
    b = float(params.get("elt_0_1", 0.0))
    c = float(params["elt_0_2"])
    d = float(params.get("elt_1_0", 0.0))
    e = float(params["elt_1_1"])
    f = float(params["elt_1_2"])
    NATIVE_TRANSFORM = [a, b, c, d, e, f]
else:
    # Already a list like [a,b,c,d,e,f]
    NATIVE_TRANSFORM = list(t)

print("Native CRS:", NATIVE_CRS)
print("Native transform:", NATIVE_TRANSFORM)

Native CRS: EPSG:4326
Native transform: [0.1, 0.0, -180.05, 0.0, -0.1, 90.05]


## Study region: Portugal (incl. Azores + Madeira)

In [4]:
# Portugal boundary from a built-in dataset (includes Atlantic islands)
gaul0 = ee.FeatureCollection("FAO/GAUL/2015/level0")
pt_geom = gaul0.filter(ee.Filter.eq("ADM0_NAME", "Portugal")).geometry()

# Mask image (cheap to apply)
pt_mask = ee.Image.constant(1).clip(pt_geom).selfMask()

# Export region in the same CRS as the export
# True rectangle: west, south, east, north (in degrees, EPSG:4326)
b = pt_geom.bounds(1)  # maxError=1 meter reduces weird densification
coords = ee.List(ee.List(b.coordinates().get(0)))

# coords are [ [x0,y0], [x1,y1], [x2,y2], [x3,y3], [x0,y0] ]
xs = coords.map(lambda p: ee.List(p).get(0))
ys = coords.map(lambda p: ee.List(p).get(1))

west  = ee.Number(xs.reduce(ee.Reducer.min()))
east  = ee.Number(xs.reduce(ee.Reducer.max()))
south = ee.Number(ys.reduce(ee.Reducer.min()))
north = ee.Number(ys.reduce(ee.Reducer.max()))

export_region = ee.Geometry.Rectangle([west, south, east, north], proj=None, geodesic=False)

print("Portugal geometry loaded.")
print("export_region:", export_region.getInfo())
print("native crs:", native_proj.crs().getInfo())

Portugal geometry loaded.
export_region: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-9.495945563862705, 36.96125673583286], [-6.1826907958597195, 36.96125673583286], [-6.1826907958597195, 42.14564840233552], [-9.495945563862705, 42.14564840233552], [-9.495945563862705, 36.96125673583286]]]}
native crs: EPSG:4326


## Determine ERA5-Land native projection (CRS + transform)

In [5]:
# Inspect the dataset projection / nominal scale (sanity check).
sample_proj = ee.Image(era5.first()).select(TEMP_BAND).projection()

print("Native CRS:", NATIVE_CRS)
print("Native affine transform:", NATIVE_TRANSFORM)
print("Native nominal scale (m):", sample_proj.nominalScale().getInfo())

Native CRS: EPSG:4326
Native affine transform: [0.1, 0.0, -180.05, 0.0, -0.1, 90.05]
Native nominal scale (m): 11131.949079327358


## Build daily mean temperature collection (°C)

In [6]:
# Build a DAILY mean temperature collection (°C) over the requested period.
# NOTE: This uses the ERA5-Land DAILY_AGGR product, so it is *not* iterating over hours.

start_date = ee.Date.fromYMD(START_YEAR, 1, 1)
end_date = ee.Date.fromYMD(END_YEAR + 1, 1, 1)  # exclusive

daily = (
    era5.filterDate(start_date, end_date)
    .map(lambda im: im.subtract(273.15)
         .rename("tmean_c")
         .copyProperties(im, ["system:time_start"]))
)

print("Daily images:", daily.size().getInfo())

Daily images: 1096


## Compute yearly indices and average across 2010–2012

In [7]:
# ---- Indices per year (split by metric, export-stable) ----
# Assumes you already defined:
#   - daily : ImageCollection with band "tmean_c" (NOT clipped per-image)
#   - pt_geom : Portugal geometry (from GAUL etc.)
#   - pt_mask : ee.Image.constant(1).clip(pt_geom).selfMask()
#   - export_region : pt_geom.bounds()
#
# IMPORTANT: do NOT use .clip(pt_geom) in these computations.

def hdd_for_year(year: ee.Number) -> ee.Image:
    y = ee.Number(year)
    y_start = ee.Date.fromYMD(y, 1, 1)
    y_end = y_start.advance(1, "year")
    dly = daily.filterDate(y_start, y_end).select("tmean_c")

    # Count of days with Tmean < 18C
    hdd18 = dly.map(lambda im: im.lt(18)).sum().rename("hdd_18")
    return hdd18.set({"year": y})


def cdd_for_year(year: ee.Number) -> ee.Image:
    y = ee.Number(year)
    y_start = ee.Date.fromYMD(y, 1, 1)
    y_end = y_start.advance(1, "year")
    dly = daily.filterDate(y_start, y_end).select("tmean_c")

    # Count of days with Tmean > 25C
    cdd25 = dly.map(lambda im: im.gt(25)).sum().rename("cdd_25")
    return cdd25.set({"year": y})


def extremes_for_year(year: ee.Number) -> ee.Image:
    y = ee.Number(year)
    y_start = ee.Date.fromYMD(y, 1, 1)
    y_end = y_start.advance(1, "year")
    dly = daily.filterDate(y_start, y_end).select("tmean_c")

    # Convert to array image: dimensions [image, band]
    arr = dly.toArray()
    image_axis = 0
    band_axis = 1

    # Keep the (single) band as band-axis length=1
    temps = arr.arraySlice(band_axis, 0, 1)

    # Hottest 6th–10th (indices 5..9)
    sorted_desc = temps.arraySort(temps.multiply(-1))
    hottest_6_10 = sorted_desc.arraySlice(image_axis, 5, 10)
    mean_hot = (
        hottest_6_10
        .arrayReduce(ee.Reducer.mean(), [image_axis])
        .arrayProject([band_axis])
        .arrayFlatten([["extreme_heat"]])
    )

    # Coldest 6th–10th (indices 5..9)
    sorted_asc = temps.arraySort(temps)
    coldest_6_10 = sorted_asc.arraySlice(image_axis, 5, 10)
    mean_cold = (
        coldest_6_10
        .arrayReduce(ee.Reducer.mean(), [image_axis])
        .arrayProject([band_axis])
        .arrayFlatten([["extreme_cold"]])
    )

    return ee.Image.cat([mean_hot, mean_cold]).set({"year": y})


years = ee.List.sequence(START_YEAR, END_YEAR)

# Build yearly collections separately (so exporting HDD doesn't trigger extremes)
hdd_yearly = ee.ImageCollection(years.map(hdd_for_year))
cdd_yearly = ee.ImageCollection(years.map(cdd_for_year))
ext_yearly = ee.ImageCollection(years.map(extremes_for_year))

# Average across years
avg_hdd18 = hdd_yearly.mean()
avg_cdd25 = cdd_yearly.mean()
avg_extremes = ext_yearly.mean()

# Mask once at the end (cheap; avoids polygon clipping in the graph)
avg_hdd18 = avg_hdd18.updateMask(pt_mask)
avg_cdd25 = avg_cdd25.updateMask(pt_mask)
avg_extremes = avg_extremes.updateMask(pt_mask)

# Optional resampling (only after everything is computed)
if USE_BILINEAR:
    avg_hdd18 = avg_hdd18.resample("bilinear")
    avg_cdd25 = avg_cdd25.resample("bilinear")
    avg_extremes = avg_extremes.resample("bilinear")

# Convenience image (for visualization only)
avg_indices = ee.Image.cat([avg_hdd18, avg_cdd25, avg_extremes])

print("Bands:", avg_indices.bandNames().getInfo())

Bands: ['hdd_18', 'cdd_25', 'extreme_heat', 'extreme_cold']


## Export GeoTIFFs to Google Drive (native ERA5-Land grid)

In [8]:
def export_band(band_name: str):
    file_base = f"PT_{band_name}_ERA5L_skinT_dailyMean_{START_YEAR}_{END_YEAR}_nativegrid"

    if band_name == "hdd_18":
        image = avg_hdd18
    elif band_name == "cdd_25":
        image = avg_cdd25
    elif band_name in ["extreme_heat", "extreme_cold"]:
        image = avg_extremes.select([band_name])
    else:
        raise ValueError("Unknown band: " + band_name)

    # Ensure mask is applied and the export image has a stable default projection
    image = image.updateMask(pt_mask).setDefaultProjection(native_proj)

    task = ee.batch.Export.image.toDrive(
        image=image.toFloat(),
        description=f"EXPORT_{file_base}",
        folder=DRIVE_FOLDER,
        fileNamePrefix=file_base,
        region=export_region,          # your 5-point rectangle (good)
        crs=NATIVE_CRS,
        crsTransform=NATIVE_TRANSFORM, # now valid numbers
        maxPixels=MAX_PIXELS,
        fileFormat="GeoTIFF"
    )
    task.start()
    return task

bands = ["hdd_18", "cdd_25", "extreme_heat", "extreme_cold"]
tasks = {b: export_band(b) for b in bands}

print("Started export tasks:")
for b, t in tasks.items():
    print(f"  {b:13s}  task_id={t.id}")


Started export tasks:
  hdd_18         task_id=25JJJF2Q5KWHXZ6HT5BNBUCD
  cdd_25         task_id=TCE47KZR3CW3S47OADYVZY5P
  extreme_heat   task_id=JDNEBK6WZPXZSVI2BQ67JWXB
  extreme_cold   task_id=XTV6RG3PTA2TKPIBPELGNBVU


## (Optional) Poll task statuses

In [9]:
def show_status():
    for b, t in tasks.items():
        s = t.status()
        print(f"{b:13s} | {s.get('state'):10s} | {s.get('description')}")

# Poll a few times (adjust as you like)
for _ in range(6):
    show_status()
    time.sleep(10)


hdd_18        | READY      | EXPORT_PT_hdd_18_ERA5L_skinT_dailyMean_2010_2012_nativegrid
cdd_25        | READY      | EXPORT_PT_cdd_25_ERA5L_skinT_dailyMean_2010_2012_nativegrid
extreme_heat  | READY      | EXPORT_PT_extreme_heat_ERA5L_skinT_dailyMean_2010_2012_nativegrid
extreme_cold  | READY      | EXPORT_PT_extreme_cold_ERA5L_skinT_dailyMean_2010_2012_nativegrid
hdd_18        | READY      | EXPORT_PT_hdd_18_ERA5L_skinT_dailyMean_2010_2012_nativegrid
cdd_25        | READY      | EXPORT_PT_cdd_25_ERA5L_skinT_dailyMean_2010_2012_nativegrid
extreme_heat  | READY      | EXPORT_PT_extreme_heat_ERA5L_skinT_dailyMean_2010_2012_nativegrid
extreme_cold  | READY      | EXPORT_PT_extreme_cold_ERA5L_skinT_dailyMean_2010_2012_nativegrid
hdd_18        | READY      | EXPORT_PT_hdd_18_ERA5L_skinT_dailyMean_2010_2012_nativegrid
cdd_25        | READY      | EXPORT_PT_cdd_25_ERA5L_skinT_dailyMean_2010_2012_nativegrid
extreme_heat  | READY      | EXPORT_PT_extreme_heat_ERA5L_skinT_dailyMean_2010_2012_na

In [1]:
Export.image.toDrive(
    image=img,
    description="test_no_clip",
    crs=NATIVE_CRS,
    crsTransform=NATIVE_TRANSFORM,
    maxPixels=1e13
)


NameError: name 'Export' is not defined